# Project Categories Analysis  

In [1]:
import pandas as pd
from siuba import *
import numpy as np

In [8]:
import altair as alt
from shared_utils import geography_utils
from shared_utils import altair_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

In [3]:
from IPython.display import Markdown

In [4]:
import _dla_utils

In [5]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', 1000)

In [6]:
df= pd.read_parquet("dla_df.parquet")

In [7]:
df.sample()

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,process_days,primary_agency_name,adjusted_total_requested,adjusted_fed_requested,adjusted_ac_requested,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
4577,Obligated,CML,5197(030),Sanger,2017-03-29,2017-04-03,2017-04-05,2017-04-06,2017-04-17,"26,500.00",0.00,"30,000.00",Authorized,5197,6,E-76 approved on Prepared on 2/14/2017,5.00,3.00,11.00,FRE150059,Bethel Ave From North Ave To Annadale Ave,"Construct Class Ii Bike Lane, Curb, Gutter And Sidewalk",1,NaT,NaT,CFCG,5197,"2,017.00",19.00,Sanger,"33,163.81","29,294.70",0.00,1,0,0,1,0,1,0,3


In [9]:
df.work_categories.value_counts()

1    9222
2    8499
0    1998
3    1293
4      47
Name: work_categories, dtype: int64

## Group By: Prepared Year 

In [10]:
sum_groups = (df.groupby(['prepared_y'])
               .agg({'active_transp':'sum', 'transit':'sum', 'bridge':'sum',
                      'street':'sum','freeway':'sum','infra_resiliency_er':'sum',
                     'congestion_relief':'sum'}).reset_index())
sum_groups.set_index('prepared_y', inplace=True)
sum_groups

,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief
prepared_y,,,,,,,
"2,010.00",0,0,0,1,0,0,0
"2,011.00",0,0,1,3,1,2,0
"2,012.00",4,0,0,2,0,0,0
"2,013.00",39,7,36,58,2,102,0
"2,014.00",631,55,431,1045,12,1487,14
"2,015.00",600,43,455,924,23,1427,16
"2,016.00",606,54,459,952,52,1360,19
"2,017.00",599,39,420,976,25,1363,19
"2,018.00",576,55,423,1142,22,1781,28


In [11]:
sum_groups = sum_groups.append(sum_groups.sum().rename('total'))

/tmp/ipykernel_1214/1818026496.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


In [12]:
sum_groups

,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief
prepared_y,,,,,,,
2010.0,0,0,0,1,0,0,0
2011.0,0,0,1,3,1,2,0
2012.0,4,0,0,2,0,0,0
2013.0,39,7,36,58,2,102,0
2014.0,631,55,431,1045,12,1487,14
2015.0,600,43,455,924,23,1427,16
2016.0,606,54,459,952,52,1360,19
2017.0,599,39,420,976,25,1363,19
2018.0,576,55,423,1142,22,1781,28


In [13]:
pct_years = sum_groups.copy()

In [14]:
pct_years['totals'] = pct_years.sum(numeric_only=True, axis=1)

In [15]:
pct_years

,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,totals
prepared_y,,,,,,,,
2010.0,0,0,0,1,0,0,0,1
2011.0,0,0,1,3,1,2,0,7
2012.0,4,0,0,2,0,0,0,6
2013.0,39,7,36,58,2,102,0,244
2014.0,631,55,431,1045,12,1487,14,3675
2015.0,600,43,455,924,23,1427,16,3488
2016.0,606,54,459,952,52,1360,19,3502
2017.0,599,39,420,976,25,1363,19,3441
2018.0,576,55,423,1142,22,1781,28,4027


In [16]:

cols = ['active_transp', 'transit', 'bridge', 'street','freeway', 'infra_resiliency_er',
       'congestion_relief']
for col in cols: 
    pct_years[f'pct_{col}'] = (pct_years[col]/(pct_years['totals']))*100


In [17]:
pct_years

,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,totals,pct_active_transp,pct_transit,pct_bridge,pct_street,pct_freeway,pct_infra_resiliency_er,pct_congestion_relief
prepared_y,,,,,,,,,,,,,,,
2010.0,0,0,0,1,0,0,0,1,0.00,0.00,0.00,100.00,0.00,0.00,0.00
2011.0,0,0,1,3,1,2,0,7,0.00,0.00,14.29,42.86,14.29,28.57,0.00
2012.0,4,0,0,2,0,0,0,6,66.67,0.00,0.00,33.33,0.00,0.00,0.00
2013.0,39,7,36,58,2,102,0,244,15.98,2.87,14.75,23.77,0.82,41.80,0.00
2014.0,631,55,431,1045,12,1487,14,3675,17.17,1.50,11.73,28.44,0.33,40.46,0.38
2015.0,600,43,455,924,23,1427,16,3488,17.20,1.23,13.04,26.49,0.66,40.91,0.46
2016.0,606,54,459,952,52,1360,19,3502,17.30,1.54,13.11,27.18,1.48,38.83,0.54
2017.0,599,39,420,976,25,1363,19,3441,17.41,1.13,12.21,28.36,0.73,39.61,0.55
2018.0,576,55,423,1142,22,1781,28,4027,14.30,1.37,10.50,28.36,0.55,44.23,0.70


In [18]:
pct_years.drop(pct_years.tail(1).index,inplace=True) # drop last rows

In [19]:
pct_years= pct_years.reset_index()

In [136]:
def labeling(word):
    # Add specific use cases where it's not just first letter capitalized
    LABEL_DICT = {
        "prepared_y": "Year",
        "dist": "District",
        "total_requested": "Total Requested",
        "fed_requested": "Fed Requested",
        "ac_requested": "Advance Construction Requested",
        "nunique": "Number of Unique",
        "project_no": "Project Number",
        "active_transp": "Active Transportation",
        "infra_resiliency_er": "Infrastructure & Emergency Relief",
        "congestion_relief": "Congestion Relief",
    }

    if (word == "mpo") or (word == "rtpa"):
        word = word.upper()
    elif word in LABEL_DICT.keys():
        word = LABEL_DICT[word]
    else:
        word = word.replace("n_", "Number of ").title()
        word = word.replace("unique_", "Number of Unique ").title()
        word = word.replace("_", " ").title()

    return word

#### selecting just the percentages

In [23]:
pct_years= pct_years>>select(_.prepared_y, _.pct_active_transp, _.pct_transit, _.pct_bridge,
                    _.pct_street, _.pct_freeway, _.pct_infra_resiliency_er,
                    _.pct_congestion_relief)

In [24]:
pct_years = (pct_years >> gather('category', 'percent', _.pct_active_transp,
                    _.pct_transit,
                    _.pct_bridge,
                    _.pct_street,
                    _.pct_freeway,
                    _.pct_infra_resiliency_er,
                    _.pct_congestion_relief
                    ))


In [25]:
pct_years

,prepared_y,category,percent
0,"2,010.00",pct_active_transp,0.00
1,"2,011.00",pct_active_transp,0.00
2,"2,012.00",pct_active_transp,66.67
3,"2,013.00",pct_active_transp,15.98
4,"2,014.00",pct_active_transp,17.17
...,...,...,...
86,"2,018.00",pct_congestion_relief,0.70
87,"2,019.00",pct_congestion_relief,0.43
88,"2,020.00",pct_congestion_relief,0.49
89,"2,021.00",pct_congestion_relief,0.70


#### Charting

In [26]:
highlight = alt.selection(type='single', on="mouseover", 
                              fields=['category', "prepared_y"], nearest=True)
selection = alt.selection_multi(
                          fields=['category'], bind='legend')
    

chart = (alt.Chart(pct_years).mark_line().encode(
    x=alt.X('prepared_y:O', title=labeling('prepared_y')),
    y=alt.Y('percent', title=labeling('percent')),
    color=alt.Color("category:N", title = "Work Category"), 
    tooltip = alt.Tooltip(["prepared_y", 'percent', "category"]),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    size=alt.condition(~highlight, alt.value(2), alt.value(5))
    ).properties(title={
                 "text": ["Percent Breakdown of Work Catergories"]}).add_selection(
                 selection, highlight)
    )

chart = styleguide.preset_chart_config(chart)
display(chart)

alt.Chart(...)

## Totals over the Years

In [27]:
sum_groups.iloc[13]

active_transp           4897
transit                  431
bridge                  3495
street                  8614
freeway                  231
infra_resiliency_er    12441
congestion_relief        158
Name: total, dtype: int64

In [28]:
sums = sum_groups.transpose()
sums.reset_index()
# sums.rename(columns = {'index':'cat'})
sums

prepared_y,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,total
active_transp,0,0,4,39,631,600,606,599,576,586,536,604,116,4897
transit,0,0,0,7,55,43,54,39,55,52,50,61,15,431
bridge,0,1,0,36,431,455,459,420,423,419,351,406,94,3495
street,1,3,2,58,1045,924,952,976,1142,1167,1078,1031,235,8614
freeway,0,1,0,2,12,23,52,25,22,31,22,33,8,231
infra_resiliency_er,0,2,0,102,1487,1427,1360,1363,1781,1697,1446,1391,385,12441
congestion_relief,0,0,0,0,14,16,19,19,28,17,17,25,3,158


In [29]:
sums['total'].sum()

30267

#### get a total percentage over all the years

In [30]:
sums['total_pct'] = (sums['total']/(sums['total'].sum()))*100

In [31]:
sums

prepared_y,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,total,total_pct
active_transp,0,0,4,39,631,600,606,599,576,586,536,604,116,4897,16.18
transit,0,0,0,7,55,43,54,39,55,52,50,61,15,431,1.42
bridge,0,1,0,36,431,455,459,420,423,419,351,406,94,3495,11.55
street,1,3,2,58,1045,924,952,976,1142,1167,1078,1031,235,8614,28.46
freeway,0,1,0,2,12,23,52,25,22,31,22,33,8,231,0.76
infra_resiliency_er,0,2,0,102,1487,1427,1360,1363,1781,1697,1446,1391,385,12441,41.10
congestion_relief,0,0,0,0,14,16,19,19,28,17,17,25,3,158,0.52


#### select just the total numbers and percentage

In [32]:
sums_short = (sums>>select(_.total, _.total_pct)).reset_index()

In [33]:
sums_short.rename(columns={'index':'cat'}, inplace=True)

In [34]:
sums_short

prepared_y,cat,total,total_pct
0,active_transp,4897,16.18
1,transit,431,1.42
2,bridge,3495,11.55
3,street,8614,28.46
4,freeway,231,0.76
5,infra_resiliency_er,12441,41.10
6,congestion_relief,158,0.52


#### Charting

In [132]:
chart = alt.Chart(sums_short).mark_arc(innerRadius=50).encode(
    theta=alt.Theta(field="total_pct", type="quantitative"),
    color=(alt.Color(field="cat", type="nominal", title=labeling("cat"),
                     scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
                    legend=alt.Legend(title=('Project Categories'))
                    )
          )
)
chart

alt.Chart(...)

In [ ]:
melted = pd.melt(sums_short.rename({'Difference': 'Total'}, axis=1),
                 id_vars=['Year'], value_vars=['Total', 'Missing'])


## By District

In [36]:
sum_dist = (df.groupby(['dist'])
               .agg({'active_transp':'sum', 'transit':'sum', 'bridge':'sum',
                      'street':'sum','freeway':'sum','infra_resiliency_er':'sum',
                     'congestion_relief':'sum'}).reset_index())
sum_dist.set_index('dist', inplace=True)
sum_dist = sum_dist.append(sum_dist.sum().rename('total'))
sum_dist['totals'] = sum_dist.sum(numeric_only=True, axis=1)
sum_dist

/tmp/ipykernel_1214/4270876880.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,totals
dist,,,,,,,,
0,0,0,0,0,1,3,0,4
1,133,4,282,129,2,1018,0,1568
2,69,5,273,224,7,482,2,1062
3,800,21,621,950,26,1617,12,4047
4,1066,78,478,1313,39,1946,27,4947
5,166,46,349,548,4,998,4,2115
6,720,29,352,1498,29,1802,20,4450
7,822,101,299,1437,40,1498,10,4207
8,254,9,250,576,34,816,15,1954


In [37]:
dpct = sum_dist.copy()

#### get percentages by districts

In [38]:

cols = ['active_transp', 'transit', 'bridge', 'street','freeway', 'infra_resiliency_er',
       'congestion_relief']
for col in cols: 
    dpct[f'pct_{col}'] = (dpct[col]/(dpct['totals']))*100


In [39]:
dpct

,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,totals,pct_active_transp,pct_transit,pct_bridge,pct_street,pct_freeway,pct_infra_resiliency_er,pct_congestion_relief
dist,,,,,,,,,,,,,,,
0,0,0,0,0,1,3,0,4,0.00,0.00,0.00,0.00,25.00,75.00,0.00
1,133,4,282,129,2,1018,0,1568,8.48,0.26,17.98,8.23,0.13,64.92,0.00
2,69,5,273,224,7,482,2,1062,6.50,0.47,25.71,21.09,0.66,45.39,0.19
3,800,21,621,950,26,1617,12,4047,19.77,0.52,15.34,23.47,0.64,39.96,0.30
4,1066,78,478,1313,39,1946,27,4947,21.55,1.58,9.66,26.54,0.79,39.34,0.55
5,166,46,349,548,4,998,4,2115,7.85,2.17,16.50,25.91,0.19,47.19,0.19
6,720,29,352,1498,29,1802,20,4450,16.18,0.65,7.91,33.66,0.65,40.49,0.45
7,822,101,299,1437,40,1498,10,4207,19.54,2.40,7.11,34.16,0.95,35.61,0.24
8,254,9,250,576,34,816,15,1954,13.00,0.46,12.79,29.48,1.74,41.76,0.77


In [40]:

dpct = (dpct>>select( _.pct_active_transp, _.pct_transit, _.pct_bridge,
                    _.pct_street, _.pct_freeway, _.pct_infra_resiliency_er,
                    _.pct_congestion_relief))

In [41]:
#dpct = dpct.transpose()

In [42]:
dpct = dpct.reset_index()

In [43]:
dpct

,dist,pct_active_transp,pct_transit,pct_bridge,pct_street,pct_freeway,pct_infra_resiliency_er,pct_congestion_relief
0,0,0.00,0.00,0.00,0.00,25.00,75.00,0.00
1,1,8.48,0.26,17.98,8.23,0.13,64.92,0.00
2,2,6.50,0.47,25.71,21.09,0.66,45.39,0.19
3,3,19.77,0.52,15.34,23.47,0.64,39.96,0.30
4,4,21.55,1.58,9.66,26.54,0.79,39.34,0.55
5,5,7.85,2.17,16.50,25.91,0.19,47.19,0.19
6,6,16.18,0.65,7.91,33.66,0.65,40.49,0.45
7,7,19.54,2.40,7.11,34.16,0.95,35.61,0.24
8,8,13.00,0.46,12.79,29.48,1.74,41.76,0.77
9,9,14.04,5.08,2.91,40.92,0.00,37.05,0.00


In [44]:
# df_select= (dpct>>filter(_.dist==2))

In [45]:
# alt.Chart(df_select).mark_arc().encode(
#     theta=alt.Theta(field=cols, type="quantitative"),
#     color=(alt.Color(field=('pct_active_transp','pct_transit'),type="nominal", scale=alt.Scale(
#         range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS)
#                     )
#           ))

In [46]:
# dist_row = [0,1,2,3,4,5,6,7,8,9,10,11,12]

# for row in dist_row: 
    
#     cols = ['pct_active_transp', 'pct_transit', 'pct_bridge',
#                     'pct_street', 'pct_freeway', 'pct_infra_resiliency_er',
#                     'pct_congestion_relief']
    
#     df_select= (dpct>>filter(_.dist==row))
    
#     chart = alt.Chart(df_select).mark_arc().encode(
#         theta=alt.Theta(field=cols, type="quantitative"),
#         color=(alt.Color(field=cols, type="nominal", scale=alt.Scale(
#                                           range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS)
#                       #   ,
#                       #  legend=alt.Legend(title=(f'{_dla_utils.labeling(col)} Project Categories'))
#                         )
#               )
#     )
#     display(chart)

In [47]:
# dist_col = ['0', '1', '2', '3','4', '5','6','7','8','9','10','11','12']

# for col in dist_col: 

#     chart = alt.Chart(dpct).mark_arc().encode(
#         theta=alt.Theta(field=[col], type="quantitative"),
#         color=(alt.Color(field="index", type="nominal", scale=alt.Scale(
#                                           range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS)
#                       #   ,
#                       #  legend=alt.Legend(title=(f'{_dla_utils.labeling(col)} Project Categories'))
#                         )
#               )
#     )
#     display(chart)

## Top Agencies by Project Category

In [137]:
work_cat = ['active_transp', 'transit', 'bridge', 'street','freeway', 'infra_resiliency_er',
       'congestion_relief']

for i in work_cat:
    subset = (df>>filter(_[i]==1))
    subset_2 = ((_dla_utils.find_top(subset))>>filter(_.variable=='primary_agency_name')
            >>select(_.value,_.count)).head(5)
    subset_2['Percent of Category'] = (((subset_2['count'])/(len(subset)))*100)
    subset_2 =subset_2.rename(columns = {'value':'Agency', 'count':f'{_dla_utils.labeling(i)} Obligations'})
    
    
    #generate chart:
    
    subset_3= ((subset.groupby(['primary_agency_name']).agg({i:'sum',
                                                    'process_days':'mean',
                                                    'adjusted_total_requested':'mean',
                                                    'adjusted_fed_requested':'mean',
                                                    'adjusted_ac_requested':'mean'}).reset_index())>>arrange(-_[i])).head(5)
    
    subset_3 =subset_3.rename(columns = {'primary_agency_name':'Agency',
                                         'adjusted_total_requested':'Total Requested',
                                         'adjusted_fed_requested':'Fed Requested',
                                         'adjusted_ac_requested':'AC Requested'})
    
    subset_4 = pd.melt(subset_3, id_vars=['Agency'],
        value_vars=['Total Requested','Fed Requested','AC Requested'],
        var_name='Categories', value_name='value'
       )

    ## following cell block makes it hard to change name
    # subset_4 = (subset_3 >> gather('category', 'value', _.adjusted_total_requested,
    #                 _.adjusted_fed_requested,
    #                 _.adjusted_ac_requested,
    #                 ))
    
    chart = (alt.Chart(subset_4).mark_bar().encode(
        x=alt.X('value', axis=alt.Axis(format='$', title='Obligated Funding ($2021)'), scale=alt.Scale(domain=(-50000, 2000000))),
        y=alt.Y("Agency"),
        color='Categories:N',
        row='Categories:N'
    ))
    
    display(Markdown(f'**Top Agencies using {labeling(i)} Projects**'))
    display(subset_2.style.format(formatter={('Percent of Category'): "{:.2f}%"}))
    display(chart)

**Top Agencies using Active Transportation Projects**

,Agency,Active Transp Obligations,Percent of Category
0,Los Angeles,153,3.12%
1,Sacramento County,112,2.29%
2,Los Angeles County,96,1.96%
3,Fresno,94,1.92%
4,El Dorado County,92,1.88%


alt.Chart(...)

**Top Agencies using Transit Projects**

,Agency,Transit Obligations,Percent of Category
0,Los Angeles,34,7.87%
1,Stockton,22,5.09%
2,Metropolitan Transportation Commission,18,4.17%
3,Stanislaus County,17,3.94%
4,Los Angeles County,14,3.24%


...

,Agency,Infra Resiliency Er Obligations,Percent of Category
0,Humboldt County,595,4.78%
1,Santa Cruz County,414,3.32%
2,Los Angeles County,317,2.55%
3,Los Angeles,218,1.75%
4,Mendocino County,215,1.73%


alt.Chart(...)

**Top Agencies using Congestion Relief Projects**

,Agency,Congestionumber Of Relief Obligations,Percent of Category
0,Kern County Council of Governments,20,12.66%
1,Stanislaus Council of Governments,18,11.39%
2,Modesto,17,10.76%
3,San Diego Association of Governments,12,7.59%
4,Ventura County Transportation Commission,10,6.33%


alt.Chart(...)

#### Using transit to get the code

In [49]:
subset3 = (df>>filter(_.transit==1))
subset4 = ((_dla_utils.find_top(subset3))>>filter(_.variable=='primary_agency_name')
            >>select(_.value,_.count)).head(10)

In [50]:
(subset3>>count(_.primary_agency_name)>>arrange(-_.n)).head()

,primary_agency_name,n
31,Los Angeles,34
85,Stockton,22
40,Metropolitan Transportation Commission,18
84,Stanislaus County,17
32,Los Angeles County,14


In [51]:
subset4

,value,count
0,Los Angeles,34
1,Stockton,22
2,Metropolitan Transportation Commission,18
3,Stanislaus County,17
4,Los Angeles County,14
5,Yosemite Area Regional Transportation System JPA,14
6,Monterey County,12
7,Transit Joint Powers Authority For Merced County,12
8,Santa Barbara County,11
9,Tahoe Transportation District,10


In [52]:
subset4 = ((subset3.groupby(['primary_agency_name']).agg({'transit':'sum','process_days':'mean',
                                               'adjusted_total_requested':'mean',
                                               'adjusted_fed_requested':'mean',
                                               'adjusted_ac_requested':'mean'}).reset_index())>>arrange(-_.transit)).head(5)

In [53]:
subset4

,primary_agency_name,transit,process_days,adjusted_total_requested,adjusted_fed_requested,adjusted_ac_requested
31,Los Angeles,34,33.81,"2,035,854.83","930,004.26",0.00
85,Stockton,22,43.81,"431,827.69","379,428.31",0.00
40,Metropolitan Transportation Commission,18,39.00,"1,891,120.77","1,879,749.02",0.00
84,Stanislaus County,17,16.73,"114,832.95","102,106.45",0.00
32,Los Angeles County,14,36.83,"385,398.77","267,182.04","7,087.42"


In [54]:
## this did not work
# alt.Chart(subset4).mark_bar().encode(
#     x=alt.X("primary_agency_name"),
#     y=alt.Y(alt.repeat('layer'), title="Obligated Funding"),
#     color=alt.ColorDatum(alt.repeat('layer'))
# ).repeat(layer=["adjusted_total_requested", "adjusted_fed_requested", "adjusted_ac_requested"])

In [55]:
subset5 = (subset4 >> gather('category', 'value', _.adjusted_total_requested,
                    _.adjusted_fed_requested,
                    _.adjusted_ac_requested,
                    # _.process_days,
                    # _.transit
                    ))

In [56]:
subset5

,primary_agency_name,transit,process_days,category,value
0,Los Angeles,34,33.81,adjusted_total_requested,"2,035,854.83"
1,Stockton,22,43.81,adjusted_total_requested,"431,827.69"
2,Metropolitan Transportation Commission,18,39.00,adjusted_total_requested,"1,891,120.77"
3,Stanislaus County,17,16.73,adjusted_total_requested,"114,832.95"
4,Los Angeles County,14,36.83,adjusted_total_requested,"385,398.77"
5,Los Angeles,34,33.81,adjusted_fed_requested,"930,004.26"
6,Stockton,22,43.81,adjusted_fed_requested,"379,428.31"
7,Metropolitan Transportation Commission,18,39.00,adjusted_fed_requested,"1,879,749.02"
8,Stanislaus County,17,16.73,adjusted_fed_requested,"102,106.45"
9,Los Angeles County,14,36.83,adjusted_fed_requested,"267,182.04"


In [57]:
alt.Chart(subset5).mark_bar().encode(
    x=alt.X("primary_agency_name"),
    y=alt.Y('value', title="Obligated Funding"),
    color='category:N',
     column='category:N'
)

alt.Chart(...)

## Project Time-frames
- how long does it take to get a project approved for each category? 
- does this differ from the prefix codes?

In [58]:
df.sample()

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,process_days,primary_agency_name,adjusted_total_requested,adjusted_fed_requested,adjusted_ac_requested,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
6517,Obligated,HSIPL,5188(020),Chino,2016-05-24,2016-05-25,2016-06-12,2016-06-12,2016-06-22,"40,805.00",0.00,"32,780.00",Authorized,5188,8,E-76 approved on Prepared on 10/6/2015,1.00,18.00,10.00,SCAG015,Intersection Of Chino Avenue And Ramona Avenue,Upgrade Signals To Protected Left Turn Phasing,2,NaT,NaT,SCAG,5188,"2,016.00",29.00,Chino,"37,008.88","46,069.17",0.00,0,0,0,1,0,0,0,1


In [59]:
## grouping by prefix

In [60]:
time = (df.groupby(['prefix'])
               .agg({'primary_agency_name':'nunique', 'process_days':'mean', 'dist_processing_days':'mean', 'hq_processing_days':'mean',
                      'fhwa_processing_days':'mean'}).reset_index())

In [61]:
#filtering for prefixes most used by primary_agency_name and keeping top 50. 
time = (time>>arrange(-_.primary_agency_name)).head(50)

In [62]:
time>>arrange(_.process_days)

,prefix,primary_agency_name,process_days,dist_processing_days,hq_processing_days,fhwa_processing_days
315,STPLH,9,8.40,0.67,4.80,2.93
131,ESPL,16,8.66,0.74,4.00,3.97
313,STPLER,13,14.95,4.86,5.76,5.84
289,SRTSLNI,63,16.31,6.34,3.85,5.29
194,HPLU,16,19.94,4.22,5.05,10.13
156,FTACML,72,21.25,0.00,21.70,9.61
272,RPSTPL,148,22.49,10.42,7.48,4.24
204,HRRRL,40,25.82,11.02,9.43,4.87
56,CASB,8,26.08,18.54,2.54,5.50
174,FTASTPL,26,27.86,0.00,33.78,7.49


In [63]:
by_dist = (df.groupby(['dist'])
               .agg({'process_days':'mean', 'dist_processing_days':'mean', 'hq_processing_days':'mean',
                      'fhwa_processing_days':'mean'}).reset_index())

In [64]:
by_dist>>arrange(_.process_days)

,dist,process_days,dist_processing_days,hq_processing_days,fhwa_processing_days
0,0,19.00,0.00,6.00,13.00
9,9,25.23,11.69,7.06,6.22
6,6,28.15,14.10,7.40,6.38
3,3,33.77,16.15,12.93,4.72
4,4,34.88,21.49,9.39,4.19
10,10,35.45,13.64,16.51,4.82
5,5,35.69,14.89,14.39,6.53
7,7,38.29,21.12,10.27,7.26
11,11,40.01,24.67,7.65,7.50
1,1,40.88,23.62,12.29,4.23


- District 9, Bishop, and District 6, Fresno, are some of the more rural districts in the State, and have the lowest mean processing days for E-76s. 
- District 4 and 7 are scattered in the middle

In [65]:
#interested in seeing the mpo breakdown

mpos = (df.groupby(['mpo'])
               .agg({'primary_agency_name':'nunique', 'process_days':'mean', 'dist_processing_days':'mean', 'hq_processing_days':'mean',
                      'fhwa_processing_days':'mean'}).reset_index())
mpos>>arrange(_.process_days)

,mpo,primary_agency_name,process_days,dist_processing_days,hq_processing_days,fhwa_processing_days
11,MNOLTC,1,0.00,0.00,0.00,0.00
6,FSTIP,1,8.00,0.00,8.00,0.00
27,STNCOG,3,8.67,2.00,5.67,1.00
25,STACOG,10,9.00,1.50,6.90,0.60
3,COFCG,22,11.12,1.27,7.38,2.46
4,ER NONE,15,13.38,6.81,3.44,3.12
20,SHASTA,1,19.00,9.00,8.00,2.00
8,KCOG,44,24.87,10.82,7.24,6.53
16,SANDAG,74,25.93,15.34,9.51,1.08
2,CFCG,24,28.07,13.83,7.76,6.17


## Project Categories by Agency Size 
(Agency Size based on obligation)

In [66]:
df.sample()

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,process_days,primary_agency_name,adjusted_total_requested,adjusted_fed_requested,adjusted_ac_requested,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
6432,Obligated,CML,5950(387),Kern County,2016-06-24,2016-06-24,2016-06-24,2016-06-24,2016-07-01,"-121,024.71",0.00,"-136,705.51",Authorized,5950,6,E-76 approved on Prepared on 10/6/2015,0.00,0.00,7.00,KER140509,Buena Vista Blvd: Union Ave To Sr 184,Pave Dirt Shoulders,3,NaT,NaT,KCOG,5950,"2,016.00",7.00,Kern County,"-154,341.62","-136,637.87",0.00,0,0,0,0,0,1,0,1


In [67]:
group = (df>>count(_.primary_agency_name)>>arrange(_.n))

In [68]:
group

,primary_agency_name,n
4,Alameda Corridor Transportation Authority,1
8,Alameda County Transportation Improvement Authority,1
9,Alameda County Waste Management Authority,1
20,Antelope Valley Transit Authority,1
89,Chula Vista Elementary School District,1
...,...,...
175,Fresno,468
486,Santa Cruz County,480
277,Los Angeles County,483
276,Los Angeles,506


In [69]:
group.describe()

,n
count,619.00
mean,34.01
std,63.71
min,1.00
25%,6.00
50%,14.00
75%,35.00
max,710.00


In [70]:
print(group.n.quantile(0.33))
print(group.n.quantile(0.66))
print(group.n.quantile(0.9))

8.0
24.0
83.0


In [71]:
q33 = group.n.quantile(0.33).astype(float)
q66 = group.n.quantile(0.66).astype(float)

In [72]:
def fleet_size (row):
    if ((row.n > 0) and (row.n <= q33)):
        return "Small"
    elif ((row.n > q33) and (row.n <= q66)):
        return "Medium"
    elif ((row.n > q66)):
        return "Large"
    else:
        return "No Info"

In [73]:
group["agency_size"] = group.apply(lambda x: fleet_size(x), axis=1)

In [74]:
group

,primary_agency_name,n,agency_size
4,Alameda Corridor Transportation Authority,1,Small
8,Alameda County Transportation Improvement Authority,1,Small
9,Alameda County Waste Management Authority,1,Small
20,Antelope Valley Transit Authority,1,Small
89,Chula Vista Elementary School District,1,Small
...,...,...,...
175,Fresno,468,Large
486,Santa Cruz County,480,Large
277,Los Angeles County,483,Large
276,Los Angeles,506,Large


In [75]:
# make a dictonary to map the agency names to the main df

In [76]:
agency_map = dict(zip(group['primary_agency_name'], 
                          group['agency_size']))


In [77]:
df['agency_size'] = df['primary_agency_name'].map(agency_map)

In [78]:
df.sample()

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,process_days,primary_agency_name,adjusted_total_requested,adjusted_fed_requested,adjusted_ac_requested,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories,agency_size
34,Obligated,CML,5165(094),Turlock,2018-11-09,2018-11-29,2018-12-20,2018-12-20,2018-12-21,"33,840.00",0.00,"33,840.00",Authorized,5165,10,E-76 approved on,35.00,21.00,1.00,STANCOG,Intersection Of Tully Road And Tuolumne Road,"Install A Stand-alone Traffic Signal, Access Ramps, And Various Striping",1,2018-10-25,2018-10-25,STANCOG,5165,"2,018.00",57.00,Turlock,"36,516.83","36,516.83",0.00,0,0,0,1,0,0,0,1,Large


In [79]:
((df.groupby(['agency_size']).agg({'process_days':'mean',
                                   'adjusted_total_requested':'mean',
                                   'adjusted_fed_requested':'mean',
                                   'adjusted_ac_requested':'mean'}).reset_index()))

,agency_size,process_days,adjusted_total_requested,adjusted_fed_requested,adjusted_ac_requested
0,Large,38.19,"1,082,726.51","670,093.23","48,743.58"
1,Medium,39.16,"788,418.52","625,472.46","7,868.98"
2,Small,32.74,"638,717.08","528,074.71","6,304.76"


In [80]:
## looking at the category breakdowns by fleet size

In [81]:
work_cat = ['active_transp', 'transit', 'bridge', 'street','freeway', 'infra_resiliency_er',
       'congestion_relief']

for i in work_cat:
    subset = (df>>filter(_[i]==1))

    #generate chart:
    
    subset_3= ((subset.groupby(['agency_size']).agg({i:'sum',
                                                     'process_days':'mean',
                                                     'adjusted_total_requested':'mean',
                                                     'adjusted_fed_requested':'mean',
                                                     'adjusted_ac_requested':'mean'}).reset_index())>>arrange(-_[i])).head(5)
    
    subset_3 =subset_3.rename(columns = {'agency_size':'Agency Size',
                                         'adjusted_total_requested':'Total Requested',
                                         'adjusted_fed_requested':'Fed Requested',
                                         'adjusted_ac_requested':'AC Requested'})
    
    subset_4 = pd.melt(subset_3, id_vars=['Agency Size'],
        value_vars=['Total Requested','Fed Requested','AC Requested'],
        var_name='Categories', value_name='value'
       )
    
    chart = (alt.Chart(subset_4).mark_bar().encode(
        x=alt.X('value', axis=alt.Axis(format='$', title='Obligated Funding ($2021)'), scale=alt.Scale(domain=(-5000, 10000000))),
        y=alt.Y("Agency Size"),
        color='Categories:N',
        row='Categories:N'
    ))
    
    display(Markdown(f'**Breakdown {_dla_utils.labeling(i)} Projects**'))
    display(subset_3.style.format(formatter={('process_days'): "{:.2f}",
                                             ('Total Requested'): "${:.2f}",
                                             ('Fed Requested'): "${:.2f}",
                                             ('AC Requested'): "${:.2f}"}))
    display(chart)

**Breakdown Active Transp Projects**

,Agency Size,active_transp,process_days,Total Requested,Fed Requested,AC Requested
0,Large,3743,33.84,$503021.43,$337051.19,$-2422.16
1,Medium,924,38.06,$507226.80,$273770.56,$-2491.32
2,Small,229,27.52,$622995.06,$397299.47,$6.96


alt.Chart(...)

**Breakdown Transit Projects**

,Agency Size,transit,process_days,Total Requested,Fed Requested,AC Requested
0,Large,310,34.66,$803765.72,$561186.77,$9925.78
1,Medium,97,33.52,$994382.19,$346608.87,$9.28
2,Small,25,25.96,$354939.78,$251924.55,$0.00


...

,Agency Size,infra_resiliency_er,process_days,Total Requested,Fed Requested,AC Requested
0,Large,10346,39.21,$986786.83,$718966.61,$67129.55
1,Medium,1566,40.89,$1137898.18,$957395.95,$16559.23
2,Small,538,36.33,$760741.77,$649974.76,$10699.67


alt.Chart(...)

**Breakdown Congestionumber Of Relief Projects**

,Agency Size,congestion_relief,process_days,Total Requested,Fed Requested,AC Requested
0,Large,121,27.70,$896154.52,$862727.35,$1605.28
1,Medium,31,24.78,$185992.98,$182251.77,$0.00
2,Small,6,10.20,$23033.54,$16515.52,$0.00


alt.Chart(...)

### Other Metrics for Agency Size

In [82]:
from shared_utils import geography_utils

In [83]:
_dla_utils.get_nunique(df, 'primary_agency_name', 'agency_size')

,agency_size,n
2,Small,215
0,Large,208
1,Medium,196


In [98]:
(geography_utils.aggregate_by_geography(
    df,
    group_cols = ['agency_size'],
    count_cols=['location'], #using this to see how many obliations there are
    sum_cols = ["adjusted_total_requested", "adjusted_ac_requested", "adjusted_fed_requested", 'work_categories'],
    mean_cols = ["adjusted_total_requested", "adjusted_ac_requested", "adjusted_fed_requested", 'process_days'],
    nunique_cols = ["primary_agency_name", "prefix"],
    rename_cols=True).dropna(how='all'))>>arrange(-_.agency_size)

,agency_size,adjusted_ac_requested_sum,adjusted_fed_requested_sum,adjusted_total_requested_sum,work_categories_sum,adjusted_ac_requested_mean,adjusted_fed_requested_mean,adjusted_total_requested_mean,process_days_mean,location_count,prefix_nunique,primary_agency_name_nunique
2,Small,"5,737,331.18","480,019,907.66","581,232,544.98","1,232.00","6,304.76","528,074.71","638,717.08",32.74,912.00,78.00,215.00
1,Medium,"23,543,986.74","1,871,413,596.68","2,358,948,224.51","4,394.00","7,868.98","625,472.46","788,418.52",39.16,"3,003.00",131.00,196.00
0,Large,"835,367,557.60","11,484,057,826.33","18,554,684,211.77","24,657.00","48,743.58","670,093.23","1,082,726.51",38.19,"17,140.00",282.00,208.00


In [110]:
years_df = ((geography_utils.aggregate_by_geography(
    df,
    group_cols = ['agency_size', 'prepared_y'],
    count_cols=['location'], #using this to see how many obliations there are
    sum_cols = ["adjusted_total_requested", "adjusted_ac_requested", "adjusted_fed_requested", 'work_categories'],
    mean_cols = ["adjusted_total_requested", "adjusted_ac_requested", "adjusted_fed_requested", 'process_days'],
    nunique_cols = ["primary_agency_name", "prefix"],
    rename_cols=True)).dropna())>>arrange(-_.agency_size)

In [111]:
years_df

,agency_size,prepared_y,adjusted_ac_requested_sum,adjusted_fed_requested_sum,adjusted_total_requested_sum,work_categories_sum,adjusted_ac_requested_mean,adjusted_fed_requested_mean,adjusted_total_requested_mean,process_days_mean,location_count,prefix_nunique,primary_agency_name_nunique
2,Small,"2,018.00","-128,332.25","50,825,266.35","16,391,679.62",144.00,"-1,258.16","498,286.93","160,702.74",50.35,102.00,29.00,75.00
8,Small,"2,017.00","131,466.86","76,424,549.20","94,604,959.42",133.00,"1,413.62","821,769.35","1,017,257.63",31.75,93.00,24.00,67.00
11,Small,"2,016.00",0.00,"60,712,885.78","75,530,200.55",163.00,0.00,"532,569.17","656,784.35",36.82,115.00,31.00,97.00
12,Small,"2,015.00","-695,096.81","42,090,184.48","55,187,588.13",190.00,"-4,543.12","275,099.24","360,703.19",19.71,153.00,31.00,101.00
16,Small,"2,014.00","604,352.81","72,007,407.85","114,847,818.69",199.00,"3,753.74","447,250.98","713,340.49",24.43,161.00,36.00,108.00
23,Small,"2,013.00","93,054.05","3,598,618.88","4,940,208.31",7.00,"11,631.76","449,827.36","617,526.04",117.25,8.00,6.00,7.00
27,Small,"2,019.00","436,086.40","24,473,928.26","36,808,835.69",130.00,"5,012.49","281,309.52","423,090.07",49.32,87.00,27.00,64.00
29,Small,"2,020.00","1,787,088.12","77,911,145.37","100,637,998.60",125.00,"19,011.58","828,841.97","1,070,617.01",31.49,94.00,20.00,60.00
33,Small,"2,021.00","3,508,711.99","56,912,330.42","61,873,782.49",123.00,"41,278.96","669,556.83","727,926.85",37.19,85.00,27.00,61.00
38,Small,"2,022.00",0.00,"15,063,591.06","20,409,473.50",16.00,0.00,"1,255,299.26","1,700,789.46",40.43,12.00,8.00,11.00


In [113]:
highlight = alt.selection(type='single', on="mouseover", 
                              fields=['agency_size', "prepared_y"], nearest=True)
selection = alt.selection_multi(
                          fields=['agency_size'], bind='legend')
    

chart = (alt.Chart(years_df).mark_line().encode(
    x=alt.X('prepared_y:O', title=labeling('prepared_y')),
    y=alt.Y('adjusted_total_requested_mean:Q', title=('Total Requested Mean (in $2021)')),
    color=alt.Color("agency_size:N", title = "Agency Size", sort=['Small', 'Medium', 'Large']), 
    tooltip = alt.Tooltip(["prepared_y", 'adjusted_total_requested_mean', "agency_size"]),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    size=alt.condition(~highlight, alt.value(2), alt.value(5))
    ).properties(title={
                 "text": ["Average Total Requested Funds by Year"]}).add_selection(
                 selection, highlight)
    )

chart = styleguide.preset_chart_config(chart)
display(chart)

alt.Chart(...)